In [9]:
import requests
import numpy as np
from math import log, exp
from scipy.optimize import linear_sum_assignment
import time

In [10]:
url = "https://api.twelvedata.com/price"

personal = "ef3e1be5126a41c0a9d98b02f4a7ce38"
uni = "b86172a213a240fca42701a5e89e0dbe"

def get_price(curr1, curr2, key):
    if curr1 == curr2:
        return 1
    else:
        try:
            resp = requests.request("GET", url=f"{url}?symbol={curr1}/{curr2}&apikey={key}").json()['price']
            return float(resp)
        except:
            resp = requests.request("GET", url=f"{url}?symbol={curr2}/{curr1}&apikey={key}").json()['price']
            return 1/float(resp)

In [11]:
class trader:
    def __init__(self, name, curr, home_curr, start_amount):
        self.name = name
        self.last_roi = 0
        self.curr = np.array(curr)
        self.volume = self.make_volume(home_curr, start_amount)
        self.fees = 0

    def make_volume(self, home_curr, start_amount):
        volume = dict()
        for c in self.curr:
            volume[c] = 0
        volume[home_curr] = start_amount
        return volume
    
    def get_M(self):

        self.P = np.matrix([[0.0 for _ in self.curr]for _ in self.curr], dtype=float)
        self.M = np.matrix([[0.0 for _ in self.curr]for _ in self.curr], dtype=float)
        C = np.matrix([[False for _ in self.curr]for _ in self.curr], dtype=bool)

        for r, curr1 in enumerate(self.curr):
            for c, curr2 in enumerate(self.curr):
                if C[r, c] == False:
                    p = get_price(curr1, curr2, self.key)
                    if p != 0:
                        self.P[r, c] = float(p) 
                        self.P[c, r] = float(1/p)
                        self.M[r, c] = log(float(p))
                        self.M[c, r] = log(float(1/p))
                    else:
                        self.P[r, c] = 1
                        self.P[c, r] = 1
                        self.M[r, c] = 0.0
                        self.M[c, r] = 0.0
                    C[r, c] = C[c, r] = True  

        return self.M

    def set_key(self, key):
        self.key = key

    def trade(self):
        self.get_M()
        row_ind, col_ind = linear_sum_assignment(self.M, maximize=True)
        
        self.last_roi = exp(self.M[row_ind, col_ind].sum())
        
        if self.last_roi > 1 + self.fees:
            child = {k:v for k,v in zip(row_ind, col_ind)}

            visited = set()
            self.orders(child, visited)

    
    def orders(self, child, visited, p = 0):
        if p not in visited:
            visited.add(p)

            # print(f"{self.curr[p]}: {self.volume[self.curr[p]]}", end = " -> ")

            self.volume[self.curr[child[p]]] += self.volume[self.curr[p]] * self.P[p, child[p]]
            self.volume[self.curr[p]] = 0

            # print(f"{self.curr[child[p]]}: {self.volume[self.curr[child[p]]]}")

            self.orders(child, visited, child[p])
        return



In [12]:

AUD = [1000]
hunter = trader("hunter", ["AUD", "BTC", "ETH", "USD"], 'AUD', AUD[0])
hunter.set_key(personal)

start = time.time()
print(time.strftime("%H:%M:%S"), "> initial investment (AUD):", AUD[0])

i = 0
while i < 10:
    hunter.trade()
    print(f"+{str(time.time() - start)[:5]} > +{hunter.volume['AUD'] - AUD[-1]}", "total:", hunter.volume['AUD'])
    AUD.append(hunter.volume['AUD'])
    time.sleep(60)
    i += 1

14:16:18 > initial investment (AUD): 1000
+3.378 > +8.434008632677774 total: 1008.4340086326778
+66.98 > +10.711834150427194 total: 1019.145842783105
+131.0 > +7.102344522424232 total: 1026.2481873055292
+195.5 > +2.4033740889933597 total: 1028.6515613945226
+258.8 > +2.2676252176563594 total: 1030.919186612179
+322.2 > +2.3865287574128615 total: 1033.3057153695918
+385.5 > +2.311906050297466 total: 1035.6176214198892
+448.9 > +2.9985858922032094 total: 1038.6162073120925
+512.6 > +9.277475241147386 total: 1047.8936825532398
+575.9 > +8.589525453299984 total: 1056.4832080065398
